In [ ]:
#import all neccesary libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
from scipy.stats import norm
import numpy as np
%config Completer.use_jedi = False      #active autocomplete in jupyter

## Exploring dataset at high level

In [ ]:
#Load the data and visualizate it at high level
netflix_data=pd.read_csv('../input/netflix-shows/netflix_titles.csv')
netflix_data=netflix_data[netflix_data['type']=='Movie']
netflix_data.head(2)


In [ ]:
netflix_data.dtypes

### Describing all the variables within the dataset

In [ ]:
#As all variables are objetcs, let's keep exploring that at a high level
netflix_data.describe(include='object')
#We can already appreciate some inconsistencies within the dataset that may require some treatment depending on the typ
#of analysis that we want to perform.

### **We can already appreciate some inconsistencies within the dataset that may require some treatment depending on the  type of analysis that we want to perform.**
### **One of the things that i noticed is that liste_in column contains several movies subjetcs, let's split that in a few extra columns to perform some analysis.**

### **Preparing extra columns for further analysis and visualizations**

#### Creating 3 extra columns from listed_in as it contains several subjects.
#### Then, those columns are append as rows for the dataset, so each id now has 4 observations.

In [ ]:
#want to know maximum of listed in items to split columns
data=netflix_data['listed_in'].str.count(',')
max(data)
#there are max(data)+1 total items
netflix_data2=netflix_data.copy()
netflix_data2[['1st_listed','2nd_listed','3rd_listed']]=netflix_data['listed_in'].str.split(",",expand=True)
#df_test=netflix_data.copy()
col_list=['1st_listed','2nd_listed','3rd_listed']
num_columns=len(col_list)
new_df=netflix_data2.drop(col_list,axis=1)
new_df=new_df.drop('description',axis=1)
for t in range(0,len(netflix_data)):
                       for k in range(1,len(col_list)+1):
                            new_row=netflix_data2.iloc[t:t+1,:len(netflix_data2.columns)-k+1:len(netflix_data2.columns)-k]
                            new_row_info=new_df.iloc[t:t+1,:].values.tolist()[0]
                            new_item=new_row.values.tolist()[0]
                            new_row_info.append(new_item[-1])
                            del(new_row_info[-2])
                            row_fixed={new_df.columns.values[i]:new_row_info[i] for i in range(0,len(new_row_info))}
                            new_df=new_df.append(row_fixed,ignore_index=True)
                            

In [ ]:
new_df[new_df['show_id']=='s2'].head(4)

## Exploring the dataset through visualizations

In [ ]:
movies_per_year=netflix_data['release_year'].value_counts().reset_index()
movies_per_year.columns.values[0]='Year'
movies_per_year.columns.values[1]='Count'
plt.figure(figsize=[20,8])
movies_per_year2=movies_per_year[movies_per_year['Year']<2019].copy()
movies_per_year2.sort_values(by='Year',inplace=True)
sns.barplot(data=movies_per_year2,x='Year',y='Count')
plt.title('Movies produce by year',fontsize=16)
plt.xlabel('Year')
plt.ylabel('Movies_Per_Year')
plt.xticks(rotation=90)
plt.show()

### **As we can see, movies produce by year has been growing exponentially**

In [ ]:
new_df_nonull=new_df[(new_df['listed_in'] !="")& (new_df['type']=='Movie')].iloc[5377:,:].copy()                
moviestype_by_year=new_df_nonull[['listed_in','release_year']].copy()
moviestype_by_year['Counts'] = np.zeros(len(moviestype_by_year))
moviestype_by_year['listed_in']=moviestype_by_year['listed_in'].str.strip()
moviestype_by_year2=moviestype_by_year.groupby(['listed_in','release_year']).count().reset_index()
hex_colors=['#C1E1A7','#F9C1BD','#06ABC6','#622D24','#ABB5CD','#057129','#003366','#DDDDDD','#000000','#5B9BD5','#7030A0','#A2AE02','#00B050','#FFFF99','#FF3300','#FF66FF','#800000','#FF9900','#9900CC','#66FFFF','#FFFF00','#00FF00','#3366CC','#FFCCFF','#CC0066','#5F5F5F','#66FFCC','#B2B2B2','#0D8571','#FF5D5D','#B8ADF9','#F4DB9E','#D88C26','#0000CC','#D490FA',]
moviestype_by_year3=moviestype_by_year2[~moviestype_by_year2['listed_in'].isnull()]
moviestype_by_year3=moviestype_by_year3[(moviestype_by_year3['release_year']<=2018)]
plt.figure(figsize=[14,14])

summary=moviestype_by_year3[~moviestype_by_year3['listed_in'].str.contains("TV|Docu")].groupby('listed_in')['Counts'].sum().reset_index().sort_values('Counts',ascending=False).head()
summary.set_index('listed_in',inplace=True)
summary
sns.lineplot(x='release_year',y='Counts',hue='listed_in',
             data=moviestype_by_year3[~moviestype_by_year3['listed_in'].str.contains("TV|Docu")],
                                      palette=sns.set_palette(sns.color_palette(hex_colors[0:21]))
            ,dashes=False)
plt.table(cellText=summary.values,colWidths = [0.05]*len(summary.columns),
          rowLabels=summary.index,
          colLabels=summary.columns,
          cellLoc = 'center', rowLoc = 'left',
          loc='upper center',
          bbox=[1.15 ,0.34, 0.1, 0.2])
plt.grid(False)          
plt.title('Movies per year by Movie-Type',fontsize=16)
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.xticks(rotation=90)

In [ ]:
country2=netflix_data.dropna()
country2=country2[~country2['country'].str.contains(",")]
x=country2['country'].value_counts().reset_index()
titles=['country','Counts']
x.columns.values[1]=titles[1]
x.columns.values[0]=titles[0]
plt.figure(figsize=[14,10])
sns.barplot(x='country',y='Counts',data=x.sort_values(by='Counts',ascending=True),orientation="vertical")
plt.xticks(rotation=90)
plt.title('Movies by Country',fontsize=16)
plt.grid(False)
plt.show()

### **It's not a surprise for no one that USA is the one that has produced more within, almost duplicating the second highest producer, which is India.**

In [ ]:
collab=netflix_data.dropna()
collab=collab[collab['type']=='Movie']
collaborations=collab[collab['country'].str.contains(",")]
colnames=['coll_'+str(i) for i in range(0,8)]
collaborations[colnames]=collaborations['country'].str.split(",",expand=True)
x2=collaborations.groupby(colnames[0], as_index=False)[colnames[0]].size()
for i in range(1,len(colnames)):
    to_append=collaborations.groupby(colnames[i], as_index=False)[colnames[i]].size()
    to_append.columns.values[0]='coll_0'
    x2=x2.append(to_append)

plt.figure(figsize=[30,16])
x2['coll_0']=x2['coll_0'].str.strip()
x3=x2.groupby('coll_0', as_index=False)['size'].sum().sort_values('size',ascending=False)
plt.xticks(rotation=90)
plt.xlabel('Country')
plt.title('Movies collaboration per country',fontsize=22)
sns.barplot(x='coll_0',y='size',data=x3)

### **Also USA is the country that has produced more movies in collaboration with other countries, followed by UK.**

In [ ]:
netflix_data[['time_duration','mins']]=netflix_data['duration'].str.split(" ",expand=True)
netflix_data.drop(columns='mins',inplace=True)
netflix_data = netflix_data.astype({"time_duration": int})
plt.figure(figsize=[10,5])
mean,std=norm.fit(netflix_data['time_duration'].dropna())
sns.histplot(x='time_duration',data=netflix_data)
plt.axvline(x=mean, color='r', linestyle='-')
plt.title('Distribution of movie duration, average is'+ ' ' + 'mean=%.1f,std=%.1f' %(mean,std))
plt.grid(False)

### **In average a movie duration is about 100 mins, plot look kind of a normal distribution so we have almost all data points within 2 standard deviations from the mean, excluding some outliers mainly on the right side of the plot.**

In [ ]:
fig, axes = plt.subplots(5, 3, figsize=(30, 20), sharey=False)
fig.suptitle('Movie duration distribution by rating', fontsize=30)
total_it=0
for i in range(0,5):
    for k in range(0,3):
        mean,std=norm.fit(netflix_data['time_duration'][netflix_data['rating']==netflix_data['rating'].unique()[total_it]].dropna())
        sns.histplot(ax=axes[i][k],x='time_duration',
                     data=netflix_data[netflix_data['rating']==netflix_data['rating'].unique()[total_it]],
                     color=hex_colors[(i+2)*(k+1)])
        axes[i][k].axvline(x=mean, color='r', linestyle='-')
        axes[i][k].set_title('Distribution of rating type:'+str(netflix_data['rating'].unique()[total_it])+' '+ 'mean=%.1f,std=%.1f' %(mean,std),fontsize=16)
        total_it+=1
    fig.tight_layout(pad=3.0)

### **From the plots we could say that in average the longer movies are the one reltade to type: **PG-13**

### **While the ones with highest average distance between each dot and the mean (variance) are  **NC-17******